# Selection Bias Test

In [1]:
import numpy as np
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt

In [2]:
pfd = pd.read_csv("../data/pfd_clean/PFD_trans.csv")
pfd

C:\Users\Yiyan\AppData\Local\Temp/ipykernel_34104/4202235598.py:1: DtypeWarning: Columns (3,8,11,13,24) have mixed types. Specify dtype option on import or set low_memory=False.
  pfd = pd.read_csv("../data/pfd_clean/PFD_trans.csv")


,ID,Chamber,CID,CalendarYear,ReportType,Asset4SJD,Asset4Transacted,Orgname,Ultorg,RealCode,...,Asset4Purchased,Asset4Sold,Asset4Exchanged,Asset4Date,Asset4DateText,Asset4TransAmt,Asset4ExactAmt,CofD,TransNotes,Dupe
0,A170015889,s,N00009888,17,Y,S,Tortoise MLP & Pipeline Instl,Tortoise MLP & Pipeline Fund,NaN,NaN,...,x,,,NaN,09/29/2017,C,NaN,,--,d
1,A170015888,s,N00009888,17,Y,S,Vanguard Dividend Appreciation ETF,Vanguard Dividend Appreciation ETF,NaN,NaN,...,x,,,NaN,07/19/2017,C,NaN,,--,d
2,A170015887,s,N00009888,17,Y,S,UBS ETRACS Alerian MLP Infras ETN,UBS ETRACS Alerian MLP Infras ETN,NaN,NaN,...,x,,,NaN,07/19/2017,C,NaN,,--,d
3,A170015886,s,N00009888,17,Y,S,Tortoise MLP & Pipeline Instl,Tortoise MLP & Pipeline Fund,NaN,NaN,...,x,,,NaN,07/17/2017,C,NaN,,--,d
4,A170015885,s,N00009888,17,Y,S,DTC US Active Equity Common Trust Fund,DTC US Active Equity Common Trust Fund,NaN,NaN,...,x,,,NaN,07/07/2017,E,NaN,,--,d
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
377663,C160015720,s,N00030245,16,A,S,Oakmark Equity And Income Investor,Oakmark Equity & Income Fund,NaN,NaN,...,,x,,NaN,10/31/2016,B,NaN,,Sale (Full),d
377664,C160015719,s,N00030245,16,A,S,Oakmark Equity And Income Investor,Oakmark Equity & Income Fund,NaN,NaN,...,,x,,NaN,03/28/2016,B,NaN,,Sale (Partial),d
377665,C160015718,s,N00030245,16,A,S,First Eagle Global I,First Eagle Global Fund,NaN,NaN,...,,x,,NaN,10/31/2016,B,NaN,,Sale (Partial),d
377666,C160015717,s,N00030245,16,A,S,First Eagle Global I,First Eagle Global Fund,NaN,NaN,...,,x,,NaN,03/28/2016,B,NaN,,Sale (Partial),d


In [3]:
crp_id = pd.read_csv("../data/pfd/CRP_IDs.csv")
crp_id

,CID,CRPName,Party,Office,FECCandID,Congress
0,N00035451,"Adams, Alma",D,NC12,H4NC12100,117
1,N00003028,"Aderholt, Robert B",R,AL04,H6AL04098,117
2,N00033997,"Aguilar, Pete",D,CA31,H2CA31125,117
3,N00033720,"Allen, Richard W",R,GA12,H2GA12121,117
4,N00040989,"Allred, Colin",D,TX32,H8TX32098,117
...,...,...,...,...,...,...
6593,N00009824,"Wu, David",D,OR01,H8OR01072,106
6594,N00007724,"Wyden, Ron",D,ORS2,S6OR00110,106
6595,N00001849,"Wynn, Albert R",D,MD04,H2MD05361,106
6596,N00001817,"Young, C W Bill",R,FL10,H6FL06035,106


## Handles duplicates

In [4]:
crp_id_unique = crp_id[["CID", "FECCandID"]].drop_duplicates()
crp_id_unique

,CID,FECCandID
0,N00035451,H4NC12100
1,N00003028,H6AL04098
2,N00033997,H2CA31125
3,N00033720,H2GA12121
4,N00040989,H8TX32098
...,...,...
6538,N00005004,H2MO02037
6552,N00004572,H6SD00085
6567,N00004452,H6MN04103
6586,N00000340,S8RI00086


In [5]:
len(np.unique(crp_id_unique["CID"]))

1383

In [6]:
len(np.unique(crp_id_unique["FECCandID"]))

1503

In [7]:
crp_id_unique_grouped = crp_id_unique.groupby("CID").count()
crp_id_dup = np.array(crp_id_unique_grouped[crp_id_unique_grouped["FECCandID"] > 1].index)
len(crp_id_dup)

120

In [8]:
pfd_unique = pfd[~pfd["CID"].isin(crp_id_dup)]
pfd_unique.shape

(321148, 26)

In [11]:
pfd_crpid = pfd_unique.merge(crp_id_unique, on="CID", how="left")
pfd_crpid = pfd_crpid.dropna(subset=["Asset4Transacted"])
pfd_crpid

,ID,Chamber,CID,CalendarYear,ReportType,Asset4SJD,Asset4Transacted,Orgname,Ultorg,RealCode,...,Asset4Sold,Asset4Exchanged,Asset4Date,Asset4DateText,Asset4TransAmt,Asset4ExactAmt,CofD,TransNotes,Dupe,FECCandID
0,A170015889,s,N00009888,17,Y,S,Tortoise MLP & Pipeline Instl,Tortoise MLP & Pipeline Fund,NaN,NaN,...,,,NaN,09/29/2017,C,NaN,,--,d,S2TN00058
1,A170015888,s,N00009888,17,Y,S,Vanguard Dividend Appreciation ETF,Vanguard Dividend Appreciation ETF,NaN,NaN,...,,,NaN,07/19/2017,C,NaN,,--,d,S2TN00058
2,A170015887,s,N00009888,17,Y,S,UBS ETRACS Alerian MLP Infras ETN,UBS ETRACS Alerian MLP Infras ETN,NaN,NaN,...,,,NaN,07/19/2017,C,NaN,,--,d,S2TN00058
3,A170015886,s,N00009888,17,Y,S,Tortoise MLP & Pipeline Instl,Tortoise MLP & Pipeline Fund,NaN,NaN,...,,,NaN,07/17/2017,C,NaN,,--,d,S2TN00058
4,A170015885,s,N00009888,17,Y,S,DTC US Active Equity Common Trust Fund,DTC US Active Equity Common Trust Fund,NaN,NaN,...,,,NaN,07/07/2017,E,NaN,,--,d,S2TN00058
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
321143,A170015894,s,N00009888,17,Y,S,Vanguard Dividend Appreciation ETF,Vanguard Dividend Appreciation ETF,NaN,NaN,...,,,NaN,10/02/2017,C,NaN,,--,d,S2TN00058
321144,A170015893,s,N00009888,17,Y,S,UBS ETRACS Alerian MLP Infras ETN,UBS ETRACS Alerian MLP Infras ETN,NaN,NaN,...,,,NaN,10/02/2017,B,NaN,,--,d,S2TN00058
321145,A170015892,s,N00009888,17,Y,S,Vanguard 500 Index Investor,Vanguard 500 index,NaN,NaN,...,,,NaN,09/29/2017,D,NaN,,--,d,S2TN00058
321146,A170015891,s,N00009888,17,Y,S,Vanguard Extended Market Index Investor,Vanguard Extended Market Index Investor,NaN,NaN,...,,,NaN,09/29/2017,C,NaN,,--,d,S2TN00058


In [13]:
pfd_etf = pfd_crpid[pfd_crpid["Asset4Transacted"].str.contains("ETF")]
pfd_etf

,ID,Chamber,CID,CalendarYear,ReportType,Asset4SJD,Asset4Transacted,Orgname,Ultorg,RealCode,...,Asset4Sold,Asset4Exchanged,Asset4Date,Asset4DateText,Asset4TransAmt,Asset4ExactAmt,CofD,TransNotes,Dupe,FECCandID
1,A170015888,s,N00009888,17,Y,S,Vanguard Dividend Appreciation ETF,Vanguard Dividend Appreciation ETF,NaN,NaN,...,,,NaN,07/19/2017,C,NaN,,--,d,S2TN00058
122,A170017845,s,N00000491,17,Y,S,Vanguard FTSE All-Wld ex-US ETF,Vanguard FTSE All-Wld ex-US ETF,NaN,NaN,...,,,NaN,06/15/2017,B,NaN,,--,,S6ME00159
132,A180017835,s,N00000491,18,Y,S,Vanguard FTSE All-World ex-US Index Fund ETF S...,Vanguard FTSE All-World ex-US Index Fund ETF S...,NaN,NaN,...,,,01/29/2018,01/29/2018,C,NaN,,--,,S6ME00159
134,A180017090,s,N00036915,18,Y,S,iShares Core MSCI EAFE ETF,iShares Core MSCI EAFE ETF,NaN,NaN,...,x,,06/20/2018,06/20/2018,B,NaN,,Sale (Partial),d,S6CA00584
135,A180017089,s,N00036915,18,Y,S,Invesco Emerging Markets Sovereign Debt ETF,Invesco Emerging Markets Sovereign Debt ETF,NaN,NaN,...,x,,06/20/2018,06/20/2018,B,NaN,,Sale (Partial),d,S6CA00584
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
321122,A170019005,s,N00036915,17,A,S,Vanguard Short-Term Bond ETF,Vanguard Short Term Bond ETF,NaN,NaN,...,x,,NaN,04/28/2017,B,NaN,,Sale (Full),,S6CA00584
321123,A170019004,s,N00036915,17,A,S,VanEck Vectors JP Morgan EM LC Bd ETF,VanEck Vectors JP Morgan EM LC Bd ETF,NaN,NaN,...,x,,NaN,04/28/2017,B,NaN,,Sale (Full),,S6CA00584
321124,A170019003,s,N00036915,17,A,S,iShares Core MSCI Emerging Markets ETF,iShares MSCI Emerging Markets ETF,NaN,NaN,...,x,,NaN,04/28/2017,C,NaN,,Sale (Full),,S6CA00584
321129,A170015908,s,N00009888,17,Y,S,Vanguard Dividend Appreciation ETF,Vanguard Dividend Appreciation ETF,NaN,NaN,...,x,,NaN,03/02/2017,C,NaN,,Sale (Partial),d,S2TN00058


In [16]:
pfd_etf_individuals = pfd_etf[["CID", "FECCandID"]].drop_duplicates()
pfd_etf_individuals

,CID,FECCandID
1,N00009888,S2TN00058
122,N00000491,S6ME00159
134,N00036915,S6CA00584
186,N00012508,S8DE00079
522,N00001758,S0IA00028
...,...,...
287800,N00030602,H2TX33040
287999,N00038285,H6TX19099
288254,N00036154,H4NJ01084
290270,N00001692,H0DC00058


In [25]:
pfd_etf_individuals.to_csv("../data/pfd_clean/pfd_etf_individuals.csv", index=False)

## Selected Keywords

In [20]:
pfd_etf[pfd_etf["Asset4Transacted"].str.contains("Clean")]

,ID,Chamber,CID,CalendarYear,ReportType,Asset4SJD,Asset4Transacted,Orgname,Ultorg,RealCode,...,Asset4Sold,Asset4Exchanged,Asset4Date,Asset4DateText,Asset4TransAmt,Asset4ExactAmt,CofD,TransNotes,Dupe,FECCandID
12870,E140076861,H,N00033030,14,Y,,Jared's Fidelity Rollover IRA Powershares Wild...,Jared's Fidelity Rollover IRA,NaN,NaN,...,,,03/03/2014,140303,B,NaN,,NaN,,H2CA06259
12871,E140076860,H,N00033030,14,Y,,Jared's Fidelity Rollover IRA Powershares Wild...,Jared's Fidelity Rollover IRA,NaN,NaN,...,,,03/27/2014,140327,B,NaN,,NaN,,H2CA06259


In [21]:
pfd_etf[pfd_etf["Asset4Transacted"].str.contains("Sustainable")]

,ID,Chamber,CID,CalendarYear,ReportType,Asset4SJD,Asset4Transacted,Orgname,Ultorg,RealCode,...,Asset4Sold,Asset4Exchanged,Asset4Date,Asset4DateText,Asset4TransAmt,Asset4ExactAmt,CofD,TransNotes,Dupe,FECCandID


In [22]:
pfd_etf[pfd_etf["Asset4Transacted"].str.contains("Solar")]

,ID,Chamber,CID,CalendarYear,ReportType,Asset4SJD,Asset4Transacted,Orgname,Ultorg,RealCode,...,Asset4Sold,Asset4Exchanged,Asset4Date,Asset4DateText,Asset4TransAmt,Asset4ExactAmt,CofD,TransNotes,Dupe,FECCandID
4681,B160004377,S,N00027533,16,Y,J,Guggenheim Solar ETF,Guggenheim Solar ETF,NaN,NaN,...,x,,NaN,04/12/2016,B,NaN,,Sale (Full),,S6RI00221
45931,A130037558,H,N00006008,13,Y,D,Kaitlin Guggenheim Solar ETF (TAN),Kaitlin,NaN,NaN,...,X,,10/29/2013,131029,B,NaN,,NaN,,H6TX15055
45932,A130037557,H,N00006008,13,Y,D,Kaitlin Guggenheim Solar ETF (TAN),Kaitlin,NaN,NaN,...,,,10/22/2013,131022,B,NaN,,NaN,,H6TX15055
56277,E150020371,s,N00027533,15,A,J,Guggenheim Solar ETF,Guggenheim Solar ETF,NaN,NaN,...,,,11/25/2015,11/25/2015,C,NaN,,--,,S6RI00221
57262,E150018897,s,N00027533,15,Y,J,Guggenheim Solar ETF,Guggenheim Solar ETF,NaN,NaN,...,,,11/25/2015,11/25/2015,C,NaN,,--,,S6RI00221


In [23]:
pfd_etf[pfd_etf["Asset4Transacted"].str.contains("Energy")]

,ID,Chamber,CID,CalendarYear,ReportType,Asset4SJD,Asset4Transacted,Orgname,Ultorg,RealCode,...,Asset4Sold,Asset4Exchanged,Asset4Date,Asset4DateText,Asset4TransAmt,Asset4ExactAmt,CofD,TransNotes,Dupe,FECCandID
633,A170016584,s,N00033054,17,Y,,Energy Select Sector SPDR ETF,Energy Select Sector SPDR ETF,NaN,NaN,...,,,NaN,07/17/2017,B,NaN,,--,d,S2MT00096
716,A170016500,s,N00033054,17,A,,Energy Select Sector SPDR ETF,Energy Select Sector SPDR ETF,NaN,NaN,...,,,NaN,07/17/2017,B,NaN,,--,d,S2MT00096
740,A170016476,s,N00033054,17,A,J,Energy Select Sector SPDR ETF,Energy Select Sector SPDR ETF,NaN,NaN,...,x,,NaN,07/17/2017,B,NaN,,Sale (Full),d,S2MT00096
883,A170017281,s,N00033054,17,Y,,Energy Select Sector SPDR ETF,Energy Select Sector SPDR ETF,NaN,NaN,...,x,,NaN,07/17/2017,B,NaN,,Sale (Full),,S2MT00096
921,A180017197,s,N00033177,18,Y,,Vanguard Energy Index Fund ETF Shares,Vanguard Energy Index Fund,NaN,NaN,...,x,,01/12/2018,01/12/2018,C,NaN,,Sale (Partial),d,S2VA00142
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
320831,A180019296,s,N00038397,18,Y,,iShares Global Energy ETF,iShares Global Energy ETF,NaN,NaN,...,,,03/14/2018,03/14/2018,B,NaN,,--,,S6NH00091
320862,A180019268,s,N00038397,18,Y,S,iShares U.S. Energy ETF,iShares US Energy ETF,NaN,NaN,...,,,08/29/2018,08/29/2018,B,NaN,,--,,S6NH00091
320866,A180019264,s,N00038397,18,Y,S,iShares U.S. Energy ETF,iShares US Energy ETF,NaN,NaN,...,,,08/06/2018,08/06/2018,B,NaN,,--,,S6NH00091
320867,A180019263,s,N00038397,18,Y,S,iShares U.S. Energy ETF,iShares US Energy ETF,NaN,NaN,...,,,07/05/2018,07/05/2018,B,NaN,,--,,S6NH00091


In [24]:
pfd_etf[pfd_etf["Asset4Transacted"].str.contains("Green")]

,ID,Chamber,CID,CalendarYear,ReportType,Asset4SJD,Asset4Transacted,Orgname,Ultorg,RealCode,...,Asset4Sold,Asset4Exchanged,Asset4Date,Asset4DateText,Asset4TransAmt,Asset4ExactAmt,CofD,TransNotes,Dupe,FECCandID
